In [1]:
import os
import pandas as pd
import datetime

In [2]:
DATASET_PATH = os.path.join(".", "training_data_cleaned.csv")
data = pd.read_csv(DATASET_PATH)
data.head()

,Unnamed: 0,dt,weekday,year,id_driver,id_carrier_number,dim_carrier_type,dim_carrier_company_name,home_base_city,home_base_state,...,loads,marketplace_loads_otr,marketplace_loads_atlas,marketplace_loads,brokerage_loads_otr,brokerage_loads_atlas,brokerage_loads,total_loads,date,recent_date
0,0,2021-01-20,3,2021,30222,U0117613,Owner Operator,Rosas Transport,Santa Fe Springs,CA,...,4,1,253,254,0,3,3,257,2021-01-20,2021-02-17
1,1,2020-03-09,1,2020,4524,C0090405,Fleet,Daylight Transport,Long Beach,CA,...,6,0,0,0,2671,0,2671,2671,2020-03-09,2021-02-15
2,2,2020-02-14,5,2020,31164,C0097099,Fleet,First Class Transport #8,Hawthorne,CA,...,1,29,82,111,30,19,49,160,2020-02-14,2020-12-18
3,3,2019-06-10,1,2019,8859,U0097935,Owner Operator,DTELLIS TRUCKING,Long Beach,CA,...,1,25,6,31,0,1,1,32,2019-06-10,2020-02-16
4,4,2019-12-03,2,2019,18870,U0106560,Owner Operator,Z&N Transport,Los Angeles,CA,...,4,1,233,234,0,9,9,243,2019-12-03,2020-02-18


In [3]:
# drop unnecessary columns
data = data.drop('Unnamed: 0', axis=1)

## Part 1

In [4]:
# we convert the column 'most_recent_load_date' into datetime objects in order to calculate the 75th percentile 
most_recent_load_date_datetime = [datetime.datetime.strptime(x, '%Y-%m-%d') for x in data['most_recent_load_date']]
data['most_recent_load_date'] = most_recent_load_date_datetime

In [5]:
# generate labels based on 75th percentile of 'loads' and 'most_recent_load_date'

labels = []

cutoff_date = data['most_recent_load_date'].quantile(0.75)
cutoff_loads = data['loads'].quantile(0.75)

for index, row in data.iterrows():
    if(row['loads'] >= cutoff_loads and row['most_recent_load_date'] >= cutoff_date):
        labels.append(1)
    else:
        labels.append(0)
        
data['labels'] = labels

In [6]:
data['labels'].value_counts()

0    70783
1    12631
Name: labels, dtype: int64

## Part 2

In [7]:
data = data.drop(['loads', 'most_recent_load_date'], axis=1)

## Part 3

In [8]:
data.describe(include='all')

,dt,weekday,year,id_driver,id_carrier_number,dim_carrier_type,dim_carrier_company_name,home_base_city,home_base_state,carrier_trucks,...,marketplace_loads_otr,marketplace_loads_atlas,marketplace_loads,brokerage_loads_otr,brokerage_loads_atlas,brokerage_loads,total_loads,date,recent_date,labels
count,83414,83414.000000,83414.000000,83414.000000,83414,83414,83367,83369,83369,83414,...,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414.000000,83414,83414,83414.000000
unique,1907,NaN,NaN,NaN,2576,2,2494,521,40,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1907,1417,NaN
top,2021-01-14,NaN,NaN,NaN,C0094651,Fleet,NFS asset Drayage,Los Angeles,CA,"[""dryvan""]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-01-14,2021-02-17,NaN
freq,260,NaN,NaN,NaN,4845,53194,4845,17057,76146,35429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,260,11174,NaN
mean,NaN,3.500060,2018.960870,18224.058060,NaN,NaN,NaN,NaN,NaN,NaN,...,29.536241,71.511533,101.047774,148.027561,13.080166,161.107728,266.273371,NaN,NaN,0.151425
std,NaN,1.687443,1.359275,11667.851627,NaN,NaN,NaN,NaN,NaN,NaN,...,88.308416,194.548817,214.587614,415.376825,42.274194,413.197837,448.353468,NaN,NaN,0.358465
min,NaN,1.000000,2015.000000,20.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,NaN,NaN,0.000000
25%,NaN,2.000000,2018.000000,7890.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,37.000000,NaN,NaN,0.000000
50%,NaN,3.000000,2019.000000,16299.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,2.000000,0.000000,13.000000,15.000000,0.000000,37.000000,110.000000,NaN,NaN,0.000000
75%,NaN,5.000000,2020.000000,28974.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,23.000000,18.000000,94.000000,110.000000,1.000000,135.000000,325.000000,NaN,NaN,0.000000
